## MiniVGGNet CNN
This model is a lot more complex than the ShallowNet one. 

In [2]:
#imports and CUDA check
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA Available: ", torch.cuda.is_available())
print(torch.version.cuda)

CUDA Available:  True
12.1


### MiniVGGNet Model

In [ ]:
class MiniVGGNet(nn.module):
    def __init__(self, num_classes=10):
        super(MiniVGGNet, self).__init__()
        